## load training triples

In [18]:
import json
import numpy as np

with open('training_data_pairs_shuffled.json', 'r') as f:
    loaded_data = json.load(f)

In [19]:
label_list = []
anchor_list = []
game_list = []

for entry in loaded_data:
    label_list.append(entry['label'])
    anchor_list.append(f"{entry['anchor']['id']} {entry['anchor']['features']} {entry['anchor']['description']}")
    game_list.append(f"{entry['game']['id']} {entry['game']['features']} {entry['game']['description']}")

In [20]:
print(np.array(anchor_list[0]))

1 Adventure Shooter Simulator Action Fantasy Stealth actionadventure blackjack bowandarrow crime darkness death difficultylevel ghosts immersive invisibility melee movingbodies pacifistplaythrough particlesystem pickpocketing polygonald potion realtimecombat shadowstealth singleplayeronly stealthkill steampunk swimming swordplay theft thief ultimate thief back tread softly make complex non-linear levels full loot steal guards outsmart improved enemy gadgets riveting story draw world thief metal place powerful technologies fanatical religions corruption game begins garrett continues life thief however betrayed fence ambushed early mission determines truart local sheriff hunting keepers take garrett hear prophecy metal ignores garrett leaves keepers informs truart hired kill gives garrett letter directs eavesdrop mechanist meeting garrett overhears truart father karras discussing conversion street people mindless servants wear masks emit vapor capable reducing nearby humans rust truart p

In [21]:
# import polars as pl
#
# label_list = []
# anchor_id_list = []
# anchor_features_list = []
# anchor_description_list = []
# game_id_list = []
# game_features_list = []
# game_description_list = []
# 
# for entry in loaded_data:
#     label_list.append(entry['label'])
#     anchor_id_list.append(entry['anchor']['id'])
#     anchor_features_list.append(entry['anchor']['features'])
#     anchor_description_list.append(entry['anchor']['description'])
#     game_id_list.append(entry['game']['id'])
#     game_features_list.append(entry['game']['features'])
#     game_description_list.append(entry['game']['description'])
# 
# data = {
#     'label': label_list,
#     'anchor_id': anchor_id_list,
#     'anchor_features': anchor_features_list,
#     'anchor_description': anchor_description_list,
#     'game_id': game_id_list,
#     'game_features': game_features_list,
#     'game_description': game_description_list
# }
# 
# df = pl.DataFrame(data)
# df.head()

In [22]:
# all_pairs_count = df.count().item(0,0)
# print(all_pairs_count)
# train_df = df.sample(all_pairs_count*0.8, shuffle=True, seed=24)
# print(train_df.describe())

In [23]:
# corpus = train_df.get_column('anchor_description') + train_df.get_column('game_description') + train_df.get_column('anchor_features') + train_df.get_column('game_features')
# print(corpus.shape)

In [24]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# 
# MAX_NUM_WORDS = 10000
# tokenizer = Tokenizer(MAX_NUM_WORDS)
# tokenizer.fit_on_texts(corpus)
# 
# A_train_description = tokenizer.texts_to_sequences(train_df.get_column('anchor_description'))
# B_train_description = tokenizer.texts_to_sequences(train_df.get_column('game_description'))
# 
# A_train_features = tokenizer.texts_to_sequences(train_df.get_column('anchor_features'))
# B_train_features = tokenizer.texts_to_sequences(train_df.get_column('game_features'))

In [25]:
# MAX_DESC_LENGTH = max([len(d) for d in A_train_description + B_train_description])
# MAX_FEAT_LENGTH = max([len(d) for d in A_train_features + B_train_features])
# print(f'MAX_DESC_LENGTH: {MAX_DESC_LENGTH} | MAX_FEAT_LENGTH: {MAX_FEAT_LENGTH}')

In [26]:
# A_train_description = pad_sequences(A_train_description, maxlen=MAX_DESC_LENGTH)
# B_train_description = pad_sequences(B_train_description, maxlen=MAX_DESC_LENGTH)
# 
# A_train_features = pad_sequences(A_train_features, maxlen=MAX_FEAT_LENGTH)
# B_train_features = pad_sequences(B_train_features, maxlen=MAX_FEAT_LENGTH)

In [27]:
# train_df.head()

In [28]:
# new_train_df = train_df.insert_column(1, pl.Series("A_description", A_train_description))
# new_train_df = new_train_df.insert_column(1, pl.Series("B_description", B_train_description))
# new_train_df = new_train_df.insert_column(1, pl.Series("A_features", A_train_features))
# new_train_df = new_train_df.insert_column(1, pl.Series("B_features", B_train_features))

In [29]:
# new_train_df = new_train_df.drop('anchor_features', 'anchor_description', 'game_features', 'game_description')
# new_train_df.head()

In [30]:
from sklearn.model_selection import train_test_split

train_anchor, temp_anchor, train_game, temp_game, train_labels, temp_labels = train_test_split(
    anchor_list,
    game_list,
    label_list,
    test_size=0.8, random_state=42)
val_anchor, test_anchor, val_game, test_game, val_labels, test_labels = train_test_split(
    temp_anchor,
    temp_game,
    temp_labels,
    test_size=0.5, random_state=42)

In [31]:
MAX_LENGTH = max([len(d) for d in anchor_list + game_list])
NUM_CLASSES = 1
MAX_TOKENS = 10000

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

anchor_input = tf.keras.Input(shape=(1,), name='anchor', dtype=tf.string)
game_input = tf.keras.Input(shape=(1,), name='game', dtype=tf.string)

vectorize_layer = layers.TextVectorization(max_tokens=MAX_TOKENS,) 
vectorize_layer.adapt(anchor_list)

A_vectorized = vectorize_layer(anchor_input)
B_vectorized = vectorize_layer(game_input)

embedding_layer = layers.Embedding(input_dim=MAX_LENGTH, output_dim=128)
A_embedded = embedding_layer(A_vectorized)
B_embedded = embedding_layer(B_vectorized)

shared_lstm = layers.LSTM(64)
A_output = shared_lstm(A_embedded)
B_output = shared_lstm(B_embedded)

merged = layers.concatenate(
    [A_output, B_output], 
    axis=-1)

dropout1 = layers.Dropout(0.5)(merged)

dense1 = layers.Dense(
    units=NUM_CLASSES, 
    activation='relu')(dropout1)

dropout2 = layers.Dropout(0.5)(dense1)

dense2 = layers.Dense(
    units=NUM_CLASSES, 
    activation='softmax')

predictions = dense2(dropout2)

siamese_model = models.Model(inputs=[anchor_input, game_input],
                                  outputs=predictions, 
                                  name='siamese_model')

siamese_model.compile(optimizer='adam', 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])

siamese_model.summary()

In [ ]:
from keras.utils import plot_model

plot_model(siamese_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
history = siamese_model.fit(
    [np.array(train_anchor), np.array(train_game)],
    np.array(train_labels),
    epochs=10,
    batch_size=128,
    use_multiprocessing=True,
    workers=10,
    validation_data=([np.array(val_anchor), np.array(val_game)], np.array(val_labels))
)

In [ ]:
siamese_model.save('siamese_model_v2.keras')

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
test_loss, test_accuracy = siamese_model.evaluate(
    (test_anchor, test_game),
    np.array(test_labels),
)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%')